# Read clinical-trials covid dataset to set "truth"
 match to drugs in DrugBank based on Name <br>
 requires parsing out and cleansing drug-name from the <font color=blue><b>Interventions</b></font> column

### Additional Steps - for future consideration
Consider parsing <font color=blue>Title</font> field to get phase of clinical trial ?<br>
Consider using <font color=blue>URL</font> column to link to FDA web data<br>
<br>
Consider keeping <font color=blue>Status</font> and <font color=blue>Study Results</font>

In [1]:
#import other packages
import pandas as pd
import numpy as np
import os
#from itertools import chain
import time
import getpass


In [2]:
FDAPath = '/opt/sas/viya/config/data/cas/default/public/FDA/'
CTPath = FDAPath + 'ClinicalTrials/'
DBPath = FDAPath + "DrugBank/"
print(os.getcwd())
os.chdir(CTPath)
print(os.getcwd())


casenabled = False
sasid = 'sasdemo'
saspw = 'xxxxx'
if casenabled == True:
    saspw = getpass.getpass() #input()
    import swat
    s = swat.CAS("pdcesx15065.exnet.sas.com", 5570, sasid, saspw)
    #s = swat.CAS("pdcesx15065.exnet.sas.com", 5570, 'sasdemo', authinfo='/home/sasdemo/.authinfo')
    print("CAS is enabled")
else:
    print("No CAS")

/home/sasdemo05
/opt/sas/viya/config/data/cas/default/public/FDA/ClinicalTrials
No CAS


In [3]:
# http://pdcesx15065.exnet.sas.com/SASDrive 
# swat examples (including .authinfo) https://go.documentation.sas.com/?cdcId=pgmcdc&cdcVersion=8.11&docsetId=caspg3&docsetTarget=home.htm&locale=en
# https://go.documentation.sas.com/?cdcId=pgmsascdc&cdcVersion=9.4_3.5&docsetId=casref&docsetTarget=n0z3r80fjqpobvn1lvegno9gefni.htm&locale=en
# https://developer.sas.com/apis/swat/python/v1.4.0/generated/swat.cas.connection.CAS.html

if casenabled == True:
    from dlpy import Model, Sequential
    from dlpy.layers import *


In [4]:
# create process to explode columns with multiple delimited values
#  takes an array column with ['drug: drug1', 'drug: drug2', 'other: ']
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

In [5]:
display("reading: " + CTPath + 'clinicaltrial_covid.csv')
ctrials_df = pd.read_csv(CTPath + 'clinicaltrial_covid.csv')
ctrials_df.dropna(subset=['Interventions'], inplace=True) # if interventions column is blank.  drop row
display(ctrials_df.head())


'reading: /opt/sas/viya/config/data/cas/default/public/FDA/ClinicalTrials/clinicaltrial_covid.csv'

,Rank,Title,Status,Study Results,Conditions,Interventions,Locations,URL
0,1,Duvelisib to Combat COVID-19,Recruiting,No Results Available,COVID-19,Drug: Duvelisib|Procedure: Peripheral blood dr...,"Washington University School of Medicine, Sain...",https://ClinicalTrials.gov/show/NCT04372602
2,3,Decitabine for Coronavirus (COVID-19) Pneumoni...,Recruiting,No Results Available,COVID-19,Drug: Decitabine|Other: Placebo Saline,"Johns Hopkins University, Baltimore, Maryland,...",https://ClinicalTrials.gov/show/NCT04482621
4,5,A Phase 2 Trial of Infliximab in Coronavirus D...,Recruiting,No Results Available,COVID-19,Drug: Infliximab,"Tufts Medical Center, Boston, Massachusetts, U...",https://ClinicalTrials.gov/show/NCT04425538
5,6,Safety & Performance Evaluation of Seraph 100 ...,Not yet recruiting,No Results Available,COVID-19,Device: Seraph 100,"Carla Kikken-Jussen, Vaals, Limburg, Netherlands",https://ClinicalTrials.gov/show/NCT04547257
6,7,Safety and Efficacy of Tramadol in COVID-19 Eg...,Not yet recruiting,No Results Available,COVID-19,Drug: Tramadol|Other: Standard care delivered ...,NaN,https://ClinicalTrials.gov/show/NCT04454307


In [6]:
# parse title for phase


In [7]:
#ctrials_df[['Rank', 'Title', 'Interventions']].set_index(['Rank', 'Title'])['Interventions'].str.split('|')
ctrials_df.set_index(['Rank'], inplace=True)

ctrials_df['s1'] = ctrials_df['Interventions'].str.split('|')
display(ctrials_df.shape)
display(ctrials_df['Interventions'].__class__)
display(ctrials_df['s1'].__class__)
display(ctrials_df.__class__)
display(ctrials_df[['Interventions','s1']].head())


(3604, 8)

pandas.core.series.Series

pandas.core.series.Series

pandas.core.frame.DataFrame

,Interventions,s1
Rank,,
1,Drug: Duvelisib|Procedure: Peripheral blood dr...,"[Drug: Duvelisib, Procedure: Peripheral blood ..."
3,Drug: Decitabine|Other: Placebo Saline,"[Drug: Decitabine, Other: Placebo Saline]"
5,Drug: Infliximab,[Drug: Infliximab]
6,Device: Seraph 100,[Device: Seraph 100]
7,Drug: Tramadol|Other: Standard care delivered ...,"[Drug: Tramadol, Other: Standard care delivere..."


In [8]:
# now create one row for each array element in column s1
ctrials_df = explode(ctrials_df, 's1', preserve_index=True)
ctrials_df.reset_index(inplace=True)
display(ctrials_df.shape)
display(ctrials_df.head())


(5933, 9)

,index,Conditions,Interventions,Locations,Status,Study Results,Title,URL,s1
0,1,COVID-19,Drug: Duvelisib|Procedure: Peripheral blood dr...,"Washington University School of Medicine, Sain...",Recruiting,No Results Available,Duvelisib to Combat COVID-19,https://ClinicalTrials.gov/show/NCT04372602,Drug: Duvelisib
1,1,COVID-19,Drug: Duvelisib|Procedure: Peripheral blood dr...,"Washington University School of Medicine, Sain...",Recruiting,No Results Available,Duvelisib to Combat COVID-19,https://ClinicalTrials.gov/show/NCT04372602,Procedure: Peripheral blood draw
2,1,COVID-19,Drug: Duvelisib|Procedure: Peripheral blood dr...,"Washington University School of Medicine, Sain...",Recruiting,No Results Available,Duvelisib to Combat COVID-19,https://ClinicalTrials.gov/show/NCT04372602,Drug: Placebo
3,3,COVID-19,Drug: Decitabine|Other: Placebo Saline,"Johns Hopkins University, Baltimore, Maryland,...",Recruiting,No Results Available,Decitabine for Coronavirus (COVID-19) Pneumoni...,https://ClinicalTrials.gov/show/NCT04482621,Drug: Decitabine
4,3,COVID-19,Drug: Decitabine|Other: Placebo Saline,"Johns Hopkins University, Baltimore, Maryland,...",Recruiting,No Results Available,Decitabine for Coronavirus (COVID-19) Pneumoni...,https://ClinicalTrials.gov/show/NCT04482621,Other: Placebo Saline


In [9]:
new = ctrials_df["s1"].str.split(":", n = 1, expand = True) 
#display(new)

ctrials_df["iType"]= new[0].astype('str').str.strip()
ctrials_df["iName"]= new[1].astype('str').str.strip()


In [10]:
del new

# Dropping old Name columns 
ctrials_df.drop(columns =['Interventions', "s1"], inplace = True)


In [11]:
display(pd.DataFrame(ctrials_df.groupby(['iType'])['iType'].count()))


,iType
iType,
Behavioral,390
Biological,651
Combination Product,53
Device,286
Diagnostic Test,513
Dietary Supplement,138
Drug,2336
Genetic,16
Other,1384


In [12]:
# consider adding index/id of clinical trials input as a column of arrays

# keep only rows with intervention type of "Drug"
ctrials_drugs_df = ctrials_df[ ctrials_df['iType'] == 'Drug' ] 

# check results
display('ctrials_drugs_df')
display(ctrials_drugs_df.shape)

# get input row IDs for each iType + iName
print()
print('df2')
df2 = pd.DataFrame(ctrials_drugs_df[['iType', 'iName', 'index']].groupby(['iType', 'iName']).aggregate(lambda tdf: np.array(tdf.unique())))
df2.columns = ['inputRowIDs']
print(df2.shape)
print(df2['inputRowIDs'].__class__)

# counting number of row IDs for each iType + iName
Drug_iNames_df = pd.DataFrame(ctrials_drugs_df.groupby(['iType', 'iName'])['iType'].count())
Drug_iNames_df.columns = ['inputRowCount']
Drug_iNames_df['inputRowIDs'] = df2['inputRowIDs']
Drug_iNames_df.reset_index(inplace=True)
display(Drug_iNames_df.shape)
display(Drug_iNames_df[15:20])

# cleanup memory
del df2


'ctrials_drugs_df'

(2336, 9)


df2
(1313, 1)
<class 'pandas.core.series.Series'>


(1313, 4)

,iType,iName,inputRowCount,inputRowIDs
15,Drug,1% Hydrogen Peroxide,1,212
16,Drug,1% w/v Povidone-iodide,1,2295
17,Drug,1.5-2% w/v Hydrogen Peroxide,1,2295
18,Drug,10% Povidone-iodine nasal decolonization swab ...,1,830
19,Drug,100 mg/mL Virazole,2,"[1353, 1365]"


In [13]:
# consider adding index/id of clinical trials input as a column of arrays

# keep only rows with intervention type of "Drug"
ctrials_notdrugs_df = ctrials_df[ ctrials_df['iType'] != 'Drug' ] 

# check results
display('ctrials_notdrugs_df')
display(ctrials_notdrugs_df.shape)

print()
print('df2')
df2 = pd.DataFrame(ctrials_notdrugs_df[['iType', 'iName', 'index']].groupby(['iType', 'iName']).aggregate(lambda tdf: np.array(tdf.unique())))
df2.columns = ['inputRowIDs']
print(df2.shape)
print(df2['inputRowIDs'].__class__)

notDrug_iNames_df = pd.DataFrame(ctrials_notdrugs_df.groupby(['iType', 'iName'])['iType'].count())
notDrug_iNames_df.columns = ['inputRowCount']
notDrug_iNames_df['inputRowIDs'] = df2['inputRowIDs']
notDrug_iNames_df.reset_index(inplace=True)
display(notDrug_iNames_df.shape)
display(notDrug_iNames_df[:5])

# cleanup memory
del df2


'ctrials_notdrugs_df'

(3597, 9)


df2
(2898, 1)
<class 'pandas.core.series.Series'>


(2898, 4)

,iType,iName,inputRowCount,inputRowIDs
0,Behavioral,1: Prone positioning,1,2808
1,Behavioral,2: No instruction regarding positioning,1,2808
2,Behavioral,A $10 Survey Incentive,1,1228
3,Behavioral,A $20 Survey Incentive,1,1228
4,Behavioral,AMA Acknowledgement Drug Pricing,1,1677


# Load synonyms from DrugBank

In [14]:
# app.py

import json

fObj = open(DBPath+'DrugBank_CSVs/aliases.json',)
DBAlias = json.load(fObj)
fObj.close()

print(DBAlias.__class__)
display(DBAlias['DB12466'])
display(DBAlias["DB15327"])
#display(DBAlias["DB00898"]) # prednisone



<class 'dict'>


['\n      ', 'Favipiravir']

['Abivertinib']

# match intervention Names against Drug Bank to identify DB-IDs

In [15]:
# search dict in DBAlias format to return ID if element of an array matches
def searchDBAlias(byVal):
    keysList = []
    itemsList = DBAlias.items()
    for item in itemsList:
        if byVal in item[1]:
            keysList.append(item[0])
    return keysList

In [16]:
# this could be sped up if it becomes an issue
start = time.time()
Drug_iNames_df['iDBID'] = Drug_iNames_df['iName'].apply(searchDBAlias)
lapse = time.time() - start 
print("lapse time to match: ", lapse)


lapse time to match:  43.04892706871033


In [32]:
display(Drug_iNames_df[Drug_iNames_df['iDBIDCount']==1].head())


,iType,iName,inputRowCount,inputRowIDs,iDBID,iDBIDCount,iName_preclean
42,Drug,AMY-101,1,1891,[DB14803],1.0,AMY-101
49,Drug,AT-001,1,308,[DB15121],1.0,AT-001
61,Drug,AZD1222,1,14,[DB15656],1.0,AZD1222
66,Drug,Abatacept,2,"[309, 583]",[DB01281],1.0,Abatacept
69,Drug,Abivertinib,1,1352,[DB15327],1.0,Abivertinib


In [17]:
Drug_iNames_df['iDBIDCount'] = Drug_iNames_df['iDBID'].astype('str').str.count("'")/2 #.str.shape()
display(Drug_iNames_df['iDBIDCount'].value_counts())

display(Drug_iNames_df['iDBID'].__class__)
display(Drug_iNames_df['iDBID'].astype('str').__class__)
display(Drug_iNames_df['iDBID'].astype('str')[250])
display(Drug_iNames_df['iDBID'].astype('str')[400:410])

display(Drug_iNames_df[Drug_iNames_df['iDBIDCount']==2])


0.0    1078
1.0     222
2.0      12
4.0       1
Name: iDBIDCount, dtype: int64

pandas.core.series.Series

pandas.core.series.Series

"['DB00796', 'DB13919']"

400             []
401             []
402             []
403             []
404    ['DB01225']
405             []
406             []
407    ['DB08899']
408             []
409    ['DB04933']
Name: iDBID, dtype: object

,iType,iName,inputRowCount,inputRowIDs,iDBID,iDBIDCount
250,Drug,Candesartan,1,460,"[DB00796, DB13919]",2.0
349,Drug,Diphenhydramine,1,1069,"[DB01075, DB14487]",2.0
494,Drug,Hydrocortisone,3,"[454, 1069, 1831]","[DB00741, DB14539]",2.0
677,Drug,Lopinavir-Ritonavir,2,"[2642, 3035]","[DB00503, DB01601]",2.0
771,Drug,Naltrexone,3,"[890, 1292, 2587]","[DB00620, DB00704]",2.0
939,Drug,Povidone-iodine,1,809,"[DB05382, DB06812]",2.0
944,Drug,Prednisone,4,"[381, 454, 613, 1586]","[DB00635, DB00898]",2.0
1118,Drug,TNKase,1,2930,"[DB00031, DB02325]",2.0
1199,Drug,Vitamin C,5,"[1338, 2119, 2704, 3274, 4202]","[DB00126, DB14482]",2.0
1201,Drug,Vitamin D,2,"[1885, 3586]","[DB00153, DB11094]",2.0


In [18]:
start = time.time()
notDrug_iNames_df['iDBID'] = notDrug_iNames_df['iName'].apply(searchDBAlias)
lapse = time.time() - start 
print("lapse time to match: ", lapse)



lapse time to match:  88.11491131782532


In [19]:
notDrug_iNames_df['iDBIDCount'] = notDrug_iNames_df['iDBID'].astype('str').str.count("'")/2 
display(notDrug_iNames_df['iDBIDCount'].value_counts())

display(notDrug_iNames_df.groupby(['iType', 'iDBIDCount']).size().unstack(fill_value=0))
display(notDrug_iNames_df[404:410])
display(notDrug_iNames_df[notDrug_iNames_df['iDBIDCount']>1])



0.0     2854
1.0       37
2.0        5
16.0       1
7.0        1
Name: iDBIDCount, dtype: int64

iDBIDCount,0.0,1.0,2.0,7.0,16.0
iType,,,,,
Behavioral,374,1,0,0,0
Biological,454,19,0,1,1
Combination Product,50,0,0,0,0
Device,272,1,0,0,0
Diagnostic Test,480,0,0,0,0
Dietary Supplement,89,11,3,0,0
Genetic,16,0,0,0,0
Other,967,4,2,0,0
Procedure,130,1,0,0,0


,iType,iName,inputRowCount,inputRowIDs,iDBID,iDBIDCount
404,Biological,Ampion,2,"[453, 1288]",[],0.0
405,Biological,Anakinra,1,2690,[DB00026],1.0
406,Biological,Anti SARS-CoV 2 Convalescent Plasma in critica...,1,925,[],0.0
407,Biological,Anti SARS-CoV 2 Convalescent Plasma in severe ...,1,925,[],0.0
408,Biological,Anti- SARS-CoV-2 Plasma,1,3892,[],0.0
409,Biological,Anti-COVID-19 human immunoglobulin,1,1016,[],0.0


,iType,iName,inputRowCount,inputRowIDs,iDBID,iDBIDCount
552,Biological,Flucelvax,1,3736,"[DB10998, DB10999, DB11000, DB11001, DB11002, ...",7.0
553,Biological,Fluvirin,1,3736,"[DB10602, DB10780, DB10781, DB10782, DB11038, ...",16.0
1728,Dietary Supplement,Vitamin C,8,"[1104, 1233, 1823, 2015, 2344, 2718, 3033, 3061]","[DB00126, DB14482]",2.0
1729,Dietary Supplement,Vitamin D,8,"[66, 362, 815, 1038, 1978, 2344, 2944, 3033]","[DB00153, DB11094]",2.0
1738,Dietary Supplement,Zinc,4,"[2027, 2344, 3033, 3061]","[DB01593, DB14533]",2.0
2551,Other,Vitamin C,1,880,"[DB00126, DB14482]",2.0
2552,Other,Vitamin D,1,1900,"[DB00153, DB11094]",2.0


# create exception lists for review

## NOTE: any manual overrides should be entered into a .csv file and treated as data

In [20]:
Drug_iNames_df.head()

,iType,iName,inputRowCount,inputRowIDs,iDBID,iDBIDCount
0,Drug,"""Vernonia amygdalina""",1,1043,[],0.0
1,Drug,0.075% Cetylpyridinium Chloride,1,2295,[],0.0
2,Drug,0.12% Chlorhexidine Gluconate,1,2295,[],0.0
3,Drug,0.12% Chlorhexidine Gluconate Mouth Rinse,1,212,[],0.0
4,Drug,0.12% Chlorhexidine oral/nasal rinse,1,490,[],0.0


In [21]:
notDrug_iNames_df.head()

,iType,iName,inputRowCount,inputRowIDs,iDBID,iDBIDCount
0,Behavioral,1: Prone positioning,1,2808,[],0.0
1,Behavioral,2: No instruction regarding positioning,1,2808,[],0.0
2,Behavioral,A $10 Survey Incentive,1,1228,[],0.0
3,Behavioral,A $20 Survey Incentive,1,1228,[],0.0
4,Behavioral,AMA Acknowledgement Drug Pricing,1,1677,[],0.0


# Perform Cleanup Steps

### remove stop words and other logic based parsing

### I used the print at the bottom to build up lists of characters and terms that appear a large number of times but do not appear to be critical to identifying a drug's name

In [22]:
# get list of possible stop-words by breaking out every word (based on space, %, etc) into a separate word
Drug_iNames_nomatch_df = Drug_iNames_df[ Drug_iNames_df['iDBIDCount'] == 0 ].copy()


In [23]:
# a regular expression replacement would be more efficient - but this runs fast and I'm pretty quick at copy /paste
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace('"', " ")
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace("%", " ")
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace(":", " ")
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace("/", " ")
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace("-", " ")
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace("(", " ")
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace(")", " ")
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace("[", " ")
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace("]", " ")
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace("&", " ")
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace("'", " ")
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace("+", " ")
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace(",", " ")
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace("0", " ")
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace("1", " ")
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace("2", " ")
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace("3", " ")
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace("4", " ")
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace("5", " ")
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace("6", " ")
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace("7", " ")
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace("8", " ")
Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.replace("9", " ")

In [24]:

Drug_iNames_nomatch_df['iName'] = Drug_iNames_nomatch_df['iName'].str.strip()
#display(Drug_iNames_nomatch_df.head(5))

stop_words_df = Drug_iNames_nomatch_df['iName'].str.upper().str.split(" ", expand=True)
stop_words_df['iName'] = Drug_iNames_nomatch_df['iName']
#display(stop_words_df.head(5))

stop_words_df = pd.melt(stop_words_df, id_vars=['iName']) # ignore_index=False)

#print(df.replace(r'^\s*$', np.nan, regex=True))
stop_words_df["value"]= stop_words_df.value.str.replace('.', ' ')
stop_words_df["value"]= stop_words_df.value.str.replace(' +', ' ')
stop_words_df["value"]= stop_words_df.value.str.strip()
#stop_words_df["value"]= stop_words_df.value.str.replace('', np.NaN)
stop_words_df.dropna(subset=['value'], inplace=True) # if word value column is blank.  drop row
print("--" + stop_words_df.value[0] + "--")
print("--" + stop_words_df.value[1] + "--")
stop_words_df = stop_words_df[stop_words_df['value'] != ""]
stop_words_df.sort_values(by=['iName', 'variable'], ascending=True, inplace=True)
display(stop_words_df.shape)
display(stop_words_df.loc[:20])
display(stop_words_df[stop_words_df['value']=='HYDROXYCHLOROQUINE'])


#stop_words_df = pd.DataFrame(stop_words_df.groupby(['iName', 'value'])['value'].count())
stop_wordsC_df = pd.DataFrame(stop_words_df.groupby(['value'])['value'].count())
stop_wordsC_df.columns=['count']
stop_wordsC_df.reset_index(inplace=True)
stop_wordsC_df.sort_values(by='count', ascending=False, inplace=True)
display(stop_wordsC_df.shape)
display(stop_wordsC_df.loc[:5])



--VERNONIA--
----


(3391, 3)

,iName,variable,value
5391,. Cetylpyridinium Chloride,5,CETYLPYRIDINIUM
6469,. Cetylpyridinium Chloride,6,CHLORIDE
5407,. w v Hydrogen Peroxide,5,W
6485,. w v Hydrogen Peroxide,6,V
7563,. w v Hydrogen Peroxide,7,HYDROGEN
8641,. w v Hydrogen Peroxide,8,PEROXIDE
4314,. Chlorhexidine Gluconate,4,CHLORHEXIDINE
5392,. Chlorhexidine Gluconate,5,GLUCONATE
4315,. Chlorhexidine Gluconate Mouth Rinse,4,CHLORHEXIDINE
5393,. Chlorhexidine Gluconate Mouth Rinse,5,GLUCONATE


,iName,variable,value
1148,Aerolized Hydroxychloroquine Sulfate,1,HYDROXYCHLOROQUINE
1187,Apo Hydroxychloroquine,1,HYDROXYCHLOROQUINE
2295,Azithromycin and hydroxychloroquine,2,HYDROXYCHLOROQUINE
5569,Bromhexine Oral Tablet and or hydroxychloroqui...,5,HYDROXYCHLOROQUINE
2371,Chloroquine or Hydroxychloroquine,2,HYDROXYCHLOROQUINE
2372,Chloroquine or hydroxychloroquine,2,HYDROXYCHLOROQUINE
2386,Combined ART hydroxychloroquine,2,HYDROXYCHLOROQUINE
2427,Dexamethasone and Hydroxychloroquine,2,HYDROXYCHLOROQUINE
20823,Favipiravir mg mg combined with ...,19,HYDROXYCHLOROQUINE
404,Hydroxychloroquine Mg Oral Tablet,0,HYDROXYCHLOROQUINE


(1102, 2)

,value,count
648,MG,164
781,PLACEBO,83
481,HYDROXYCHLOROQUINE,65
718,OF,54
942,STANDARD,52
52,AND,51
735,ORAL,45
1023,TREATMENT,36
187,CARE,32
323,DOSE,32


In [25]:


stop_wordsC_df['conjunction'] = stop_wordsC_df.value.str.upper().isin(['IN', 'OF', 'AND', 'PLUS', 'OR', 'FOR', 'A', 'WITH'])
stop_wordsC_df['space'] = stop_wordsC_df.value.isin([' ', '.', ':', ''])
stop_wordsC_df['dose'] = stop_wordsC_df.value.str.upper().isin(['INHALATION', 'ML', 'MG', 'DOSE', 'ORAL', 'TABLETS', 'TABLET', 'ML', 'DAYS', 'INJECTION', 'KG'])
stop_wordsC_df['placebo'] = stop_wordsC_df.value.str.upper().isin(['NORMAL', 'AGENT', 'PLACEBO', 'SALINE', 'STANDARD', 'TREATMENT', 'CARE', 'THERAPY', 'NASAL', 'FLOW'])
stop_wordsC_df['concept'] = stop_wordsC_df.value.str.upper().isin(['CoV', 'COVID', 'DRUG', 'PRODUCT', 'SOLUTION', 'PLASMA'])

#termstoshow = 20
#termstoshow = 50
termstoshow = 10
display(stop_wordsC_df[(stop_wordsC_df['concept'] == False) & (stop_wordsC_df['placebo'] == False) & (stop_wordsC_df['conjunction'] == False) & (stop_wordsC_df['space'] == False) & (stop_wordsC_df['dose'] == False)].head(termstoshow))

#merge word counts back with iName and original row(s)
#write out to csv for qc work
#stop_words_df.to_csv("stop_words_candidates.csv")


,value,count,conjunction,space,dose,placebo,concept
481,HYDROXYCHLOROQUINE,65,False,False,False,False,False
884,RITONAVIR,31,False,False,False,False,False
115,AZITHROMYCIN,26,False,False,False,False,False
604,LOPINAVIR,26,False,False,False,False,False
453,HEPARIN,24,False,False,False,False,False
534,INTERFERON,24,False,False,False,False,False
559,IVERMECTIN,19,False,False,False,False,False
956,SULFATE,16,False,False,False,False,False
542,IODINE,15,False,False,False,False,False
121,B,14,False,False,False,False,False


In [26]:
# not sure of: low, agents, unfractionated
# - search the synonyms.json table for each
#   and/or
#   print out the drug - list Drug_iNames_df for iNames that contain it
display(Drug_iNames_df[Drug_iNames_df['iName'].str.upper().str.contains('LOW')])
display(Drug_iNames_df[Drug_iNames_df['iName'].str.upper().str.contains('AGENTS')])
display(Drug_iNames_df[Drug_iNames_df['iName'].str.upper().str.contains('UNFRACTIONATED')])


,iType,iName,inputRowCount,inputRowIDs,iDBID,iDBIDCount
464,Drug,Group 1: Rivaroxaban 20mg/d followed by enoxap...,1,3182,[],0.0
541,Drug,"Hydroxychloroquine, Clindamycin, Primaquine - ...",1,1962,[],0.0
691,Drug,Low Dose (10 mg) Control,1,3014,[],0.0
692,Drug,Low Molecular Weight Heparin,1,850,[],0.0
693,Drug,Low dose Interferon-beta 1a,1,1693,[],0.0
694,Drug,Low dose Low molecular weight heparin or Placebo,1,1114,[],0.0
695,Drug,Low dose prednisolone,1,1492,[],0.0
696,Drug,Low molecular weight heparin,2,"[3191, 3287]",[],0.0
697,Drug,Lower-dose prophylactic anticoagulation,1,338,[],0.0
1132,Drug,The POP02 study is collecting bodily fluid sam...,1,4217,[],0.0


,iType,iName,inputRowCount,inputRowIDs,iDBID,iDBIDCount
122,Drug,Anticoagulation Agents (Edoxaban and/or high d...,1,1114,[],0.0
123,Drug,Antihypertensive Agents,1,558,[],0.0
125,Drug,Antiviral Agents,1,2038,[],0.0
571,Drug,Immunosuppressive Agents,1,2813,[],0.0
778,Drug,Neuromuscular Blocking Agents,1,1862,[],0.0
1091,Drug,Statins (Cardiovascular Agents),1,558,[],0.0


,iType,iName,inputRowCount,inputRowIDs,iDBID,iDBIDCount
464,Drug,Group 1: Rivaroxaban 20mg/d followed by enoxap...,1,3182,[],0.0
775,Drug,Nebulised unfractionated heparin (UFH),1,1395,[],0.0
1144,Drug,Tinzaparin or unfractionated heparin,1,1476,[],0.0
1186,Drug,Unfractionated Heparin IV,1,2162,[],0.0
1187,Drug,Unfractionated heparin,2,"[42, 1135]",[],0.0
1188,Drug,Unfractionated heparin SC,1,2162,[],0.0
1189,Drug,Unfractionated heparin nebulized,1,210,[],0.0
1266,Drug,intermediate dose Enoxaparin/ unfractionated h...,1,1361,[],0.0
1299,Drug,standard prophylactic dose Enoxaparin/ unfract...,1,1361,[],0.0
1307,Drug,unfractionated Heparin,2,"[210, 1850]",[],0.0


In [27]:
# add an additional file to INSERT complicated records that contain more than one drug
# 541	Drug	Hydroxychloroquine, Clindamycin, Primaquine - ...


In [28]:
# logic parsing

# placebo - saline control

# remove stop-words - consider reading list from a file ?
#   maybe start with parsing out every word in the file from records that fail to match DrugBank

stopWords = ['0.12%', '0.075%', '100 mg']
Drug_iNamesClean_df = Drug_iNames_df
Drug_iNamesClean_df['iName_preclean'] = Drug_iNamesClean_df['iName']
display(Drug_iNamesClean_df.head())


,iType,iName,inputRowCount,inputRowIDs,iDBID,iDBIDCount,iName_preclean
0,Drug,"""Vernonia amygdalina""",1,1043,[],0.0,"""Vernonia amygdalina"""
1,Drug,0.075% Cetylpyridinium Chloride,1,2295,[],0.0,0.075% Cetylpyridinium Chloride
2,Drug,0.12% Chlorhexidine Gluconate,1,2295,[],0.0,0.12% Chlorhexidine Gluconate
3,Drug,0.12% Chlorhexidine Gluconate Mouth Rinse,1,212,[],0.0,0.12% Chlorhexidine Gluconate Mouth Rinse
4,Drug,0.12% Chlorhexidine oral/nasal rinse,1,490,[],0.0,0.12% Chlorhexidine oral/nasal rinse


### apply manual override file (for when just typing in the change is easier)

In [29]:
# read exception override file
iName_Overrides_df = pd.read_csv(CTPath + 'ctrials_iname_overrides.csv', delimiter='\t')
display(iName_Overrides_df.head())

# apply exception override file
Drug_iNamesNew_df = pd.merge(Drug_iNames_df, iName_Overrides_df, how='left', on='iName', indicator=True)
display(Drug_iNamesNew_df['_merge'].value_counts())

Drug_iNamesNew_df.rename(columns = {'iName':'iNameOld'}, inplace = True)

display(Drug_iNamesNew_df.head())


,iName,iNameNew
0,Alteplase 50 MG [Activase],Alteplase


left_only     1312
both             1
right_only       0
Name: _merge, dtype: int64

,iType,iNameOld,inputRowCount,inputRowIDs,iDBID,iDBIDCount,iName_preclean,iNameNew,_merge
0,Drug,"""Vernonia amygdalina""",1,1043,[],0.0,"""Vernonia amygdalina""",NaN,left_only
1,Drug,0.075% Cetylpyridinium Chloride,1,2295,[],0.0,0.075% Cetylpyridinium Chloride,NaN,left_only
2,Drug,0.12% Chlorhexidine Gluconate,1,2295,[],0.0,0.12% Chlorhexidine Gluconate,NaN,left_only
3,Drug,0.12% Chlorhexidine Gluconate Mouth Rinse,1,212,[],0.0,0.12% Chlorhexidine Gluconate Mouth Rinse,NaN,left_only
4,Drug,0.12% Chlorhexidine oral/nasal rinse,1,490,[],0.0,0.12% Chlorhexidine oral/nasal rinse,NaN,left_only


### now match cleaned text into drug-bank

In [30]:
# re-match to drugbank synonyms.json (note: this file could ALSO be improved)
start = time.time()
notDrug_iNames_df['iDBID'] = Drug_iNamesNew_df['iName_preclean'].apply(searchDBAlias)
lapse = time.time() - start 
print("lapse time to match: ", lapse)


In [1]:
# write out match report(s):
#Drug_iNames_df.to_csv("ctrials_Drug_iNames.csv")
#notDrug_iNames_df.to_csv("ctrials_notDrug_iNames.csv") # maybe just append to first file - drop headers ?

# maybe separate out by matched, not matched rather than Drug notDrug ?


# Write out ClinicalTrialsTruth.tsv file

DBID, iType, iName (most common? or list ?), inputRows, status, study_results, (parse phase?)

In [33]:
ProposedTruth_df = Drug_iNames_df[ Drug_iNames_df['iDBIDCount'] > 0 ].copy()
display(ProposedTruth_df.head())
display(ProposedTruth_df.iDBID.__class__)
display(ProposedTruth_df.iDBID.shape)
display(ProposedTruth_df.iDBID.iloc[1])
display(ProposedTruth_df.iDBID.iloc[1].__class__)

s = ProposedTruth_df.apply(lambda x: pd.Series(x['iDBID']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'iDBID'
#display(s)

ProposedTruth_df = ProposedTruth_df.drop('iDBID', axis=1).join(s)
display(ProposedTruth_df[10:30])

,iType,iName,inputRowCount,inputRowIDs,iDBID,iDBIDCount,iName_preclean
42,Drug,AMY-101,1,1891,[DB14803],1.0,AMY-101
49,Drug,AT-001,1,308,[DB15121],1.0,AT-001
61,Drug,AZD1222,1,14,[DB15656],1.0,AZD1222
66,Drug,Abatacept,2,"[309, 583]",[DB01281],1.0,Abatacept
69,Drug,Abivertinib,1,1352,[DB15327],1.0,Abivertinib


pandas.core.series.Series

(235,)

['DB15121']

list

,iType,iName,inputRowCount,inputRowIDs,iDBIDCount,iName_preclean,iDBID
87,Drug,Almitrine,1,2180,1.0,Almitrine,DB01430
91,Drug,Alteplase,1,3752,1.0,Alteplase,DB00009
93,Drug,Alvelestat,1,1162,1.0,Alvelestat,DB11863
94,Drug,Ambrisentan,1,804,1.0,Ambrisentan,DB06403
95,Drug,Amiodarone,1,340,1.0,Amiodarone,DB01118
96,Drug,Amlodipine,1,3746,1.0,Amlodipine,DB00381
98,Drug,Anakinra,9,"[271, 1841, 2149, 2263, 2330, 2974, 3274, 3989...",1.0,Anakinra,DB00026
108,Drug,Angiotensin 1-7,2,"[705, 2964]",1.0,Angiotensin 1-7,DB11720
109,Drug,Angiotensin II,1,2956,1.0,Angiotensin II,DB11842
119,Drug,Antibiotic,1,3730,4.0,Antibiotic,DB00626


In [ ]:
ProposedTruth_df.to_csv("ProposedTruth.csv")
